<a href="https://colab.research.google.com/github/hsakkout/genai-peft-finetune/blob/development/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project



* PEFT technique:  LoRA
  *  https://huggingface.co/docs/peft/main/en/conceptual_guides/lora
* Model:
  * model_id = "meta-llama/Llama-3.1-8B"
* Evaluation approach:
* Fine-tuning dataset:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata

# First set the token in Colab's secrets manager
# Then access it securely:
token = userdata.get('hf_personal_default')

# Login using the stored token
!huggingface-cli login --token {token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# Google Colab instance installs
# Reminder: Restart runtime after installation

!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install bert_score
!pip install wandb



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e4d981b6ddc132d6a12c51970e5fe8a347d2a0447ef028ebc56f7d57b8604510
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig  # Let's not use a pipeline, for full transparency
import torch



def setup_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenizer.padding_side = "left"

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True
        ),
        device_map="cuda",
        pad_token_id=tokenizer.pad_token_id
    )

    return model, tokenizer



In [5]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"  #  Note: this model needed Meta's license agreement and access approval

model, tokenizer = setup_model(model_name)

model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128009)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Llama

## Evaluate its performance


In [32]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnPeriod(StoppingCriteria):
    def __init__(self, tokenizer, stop_id):
        self.tokenizer = tokenizer
        self.stop_id = stop_id

    def __call__(self, input_ids, scores, **kwargs):
        return self.stop_id in input_ids[0][-1:]

class StopOnNewline(StoppingCriteria):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.newline_ids = tokenizer.encode('\n', add_special_tokens=False)

    def __call__(self, input_ids, scores, **kwargs):
        n = len(self.newline_ids)
        if n > len(input_ids[0]):
            return False
        if input_ids[0][-n:].tolist() == self.newline_ids:
            return True
        return False


stopping_criteria = StoppingCriteriaList([StopOnPeriod(tokenizer, stop_token_id),StopOnNewline(tokenizer)])




# Define bad words to avoid
bad_phrases = ['The final', 'The answer', 'The correct', 'Answer:',
    'A', 'B', 'A)', 'B)', 'Option A', 'Option B',
    'Choice A', 'Choice B', '(A)', '(B)']
bad_words_ids = tokenizer(bad_phrases, add_special_tokens=False).input_ids


# Get the token ID for the period
stop_token_id = tokenizer.encode(".", add_special_tokens=False)[-1]




In [33]:


def generate_text(model, tokenizer, prompt, max_new_tokens=20):
    # Prepare inputs
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    # Move inputs to GPU
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate text
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        stopping_criteria=stopping_criteria,
        bad_words_ids=bad_words_ids,
        no_repeat_ngram_size=3,
        temperature=0.1
    )

    # Calculate the length of the input prompt
    input_length = inputs['input_ids'].shape[1]

    # Extract only the generated tokens (excluding the prompt)
    generated_tokens = outputs[:, input_length:]

    # Decode only the generated tokens
    generated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    return generated_text

# Adjusted prompt
prompt = (
    "Definition: In each example you will read a short sentence (or two). "
    "Then, a pronoun in the text is mentioned. Your task is to choose a referent which the mentioned pronoun refers to. "
    "You will be given two options in each case and one choice should seem much more likely to you than the other.\n\n"
    "Positive Example 1 - Input: sentence: I couldn't put the saucepan on the rack because it was too tall. "
    "pronoun: it. A) the saucepan B) the rack Output: the rack\n\n"
    "Positive Example 2 - Input: sentence: Arnold greatly influenced Jackson, though he lived two centuries earlier. "
    "pronoun: he. A) jackson B) arnold Output: arnold\n\n"
    "Negative Example 1 - Input: sentence: Joe and David's uncles can still beat them at tennis, even though they are 30 years younger. "
    "pronoun: they. A) joe and david B) joe and david's uncles Output: joe and david's uncles\n\n"
    "Negative Example 2 - Input: sentence: Gaston passed the half-empty plate to Hershel because he was full. "
    "pronoun: he. A) Gaston B) Hershel Output: Hershel\n\n"
    "Now complete the following example - Input: sentence: Joe and Steve paid the detectives after they delivered the final report on the case. "
    "pronoun: they. A) joe and steve B) the detectives Output:"
)

result = generate_text(model, tokenizer, prompt, max_new_tokens=10)

print(result[0].strip())

Answer:
the detectives


In [19]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset(
    'Lots-of-LoRAs/task249_enhanced_wsc_pronoun_disambiguation',
    split='train'
)


# Print the column names
print("Dataset Columns:", dataset.column_names)

Dataset Columns: ['input', 'output', 'id']


In [34]:


from tqdm import tqdm
from rouge_score import rouge_scorer
import numpy as np
import pandas as pd




# Adjust the preprocessing function based on actual column names
def preprocess_function(examples):
    inputs = examples['input']
    raw_references = examples['output']

    prompts = []
    references = []

    for input_text, ref in zip(inputs, raw_references):
        # Build the prompt for each example
        prompt = (
            input_text.strip()
        )
        prompts.append(prompt)

        # Process the reference
        if isinstance(ref, list):
            ref = ref[0] if ref else ''
        else:
            ref = str(ref)

        references.append(ref.strip())

    return {
        'prompts': prompts,
        'references': references,
    }

data = dataset.map(preprocess_function, batched=True)

# Limit to the first example for testing (optional)
data = data.select(range(20))

# Step 4: Generate model outputs in batches
batch_size = 10  # Adjust based on your GPU memory
generated_answers = []
references = []
questions_list = []

for i in tqdm(range(0, len(data), batch_size)):
    batch = data[i:i+batch_size]
    prompts = batch['prompts']
    refs = batch['references']

    # Tokenize inputs
    inputs = tokenizer(
        prompts,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=400
    ).to('cuda')

    # Generate outputs
    outputs = model.generate(
        **inputs,
        max_new_tokens=15,  # Adjust as needed
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        stopping_criteria=stopping_criteria,
        bad_words_ids=bad_words_ids,
        no_repeat_ngram_size=3,
        temperature=0.1,
        num_beams=5,
        early_stopping=True
    )

    # **Calculate the length of the input prompt**
    input_length = inputs['input_ids'].shape[1]

    # **Extract only the generated tokens (excluding the prompt)**
    generated_tokens = outputs[:, input_length:]

    # Decode outputs
    decoded_outputs = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Post-process outputs if necessary
    processed_outputs = [output.strip() for output in decoded_outputs]

    generated_answers.extend(processed_outputs)
    references.extend(refs)
    questions_list.extend(prompts)

    # Clear CUDA cache
    torch.cuda.empty_cache()

# Minimal post-processing to clean the outputs (optional)
cleaned_outputs = []
for output in generated_answers:
    # Split at the first newline or period
    output = output.split('\n')[0].split('.')[0].strip()
    cleaned_outputs.append(output)

# Step 5: Compute evaluation metrics and store detailed results
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

detailed_results = []

for idx, (question, ref, pred) in enumerate(zip(
    questions_list, references, cleaned_outputs)):
    # Compute ROUGE scores for our model's prediction
    scores = scorer.score(ref, pred)

    # Store the detailed results
    detailed_results.append({
        'No.': idx + 1,
        'Question': question,
        'Reference': ref,
        'Our Prediction': pred,
        'Our ROUGE-1 F1': scores['rouge1'].fmeasure,
        'Our ROUGE-2 F1': scores['rouge2'].fmeasure,
        'Our ROUGE-L F1': scores['rougeL'].fmeasure,
    })

# Step 6: Create a pandas DataFrame
df = pd.DataFrame(detailed_results)

# Set pandas options for better display
pd.set_option('display.max_colwidth', None)  # Don't truncate text in cells

# Display the DataFrame in the notebook
display(df[['No.', 'Question', 'Reference', 'Our Prediction',
            'Our ROUGE-1 F1', 'Our ROUGE-2 F1', 'Our ROUGE-L F1']])

# Also print the average scores
avg_rouge1 = df['Our ROUGE-1 F1'].mean()
avg_rouge2 = df['Our ROUGE-2 F1'].mean()
avg_rougeL = df['Our ROUGE-L F1'].mean()

print(f"\nAverage ROUGE Scores for Our Model:")
print(f"ROUGE-1 F1 Score: {avg_rouge1:.4f}")
print(f"ROUGE-2 F1 Score: {avg_rouge2:.4f}")
print(f"ROUGE-L F1 Score: {avg_rougeL:.4f}")

100%|██████████| 2/2 [00:30<00:00, 15.49s/it]


,No.,Question,Reference,Our Prediction,Our ROUGE-1 F1,Our ROUGE-2 F1,Our ROUGE-L F1
0,1,"Definition: In each example you will read a short sentence (or two). Then, a pronoun in the text is mentioned. Your task is to choose a referent which the mentioned pronoun refers to. You will be given two options in each case and one choice should seem much more likely to you than the other.\n\nPositive Example 1 -\nInput: sentence: I couldn't put the saucepan on the rack because it was too tall .pronoun: it. A) the saucepan B) the rack\nOutput: the rack\n\nPositive Example 2 -\nInput: sentence: Arnold greatly influenced Jackson , though he lived two centuries earlier .pronoun: he. A) jackson B) arnold\nOutput: arnold\n\nNegative Example 1 -\nInput: sentence: Joe and David's uncles can still beat them at tennis , even though they are 30 years younger .pronoun: they. A) joe and david B) joe and david's uncles\nOutput: joe and david's uncles\n\nNegative Example 2 -\nInput: sentence: gaston passed the half-empty plate to hershel because he was full .pronoun: he. A) Gaston B) hershel\nOutput: hershel\n\nNow complete the following example -\nInput: sentence: Joe and Steve paid the detectives after they delivered the final report on the case .pronoun: they. A) joe and steve B) the detectives\nOutput:",the detectives,A),0.000000,0.000000,0.000000
1,2,"Definition: In each example you will read a short sentence (or two). Then, a pronoun in the text is mentioned. Your task is to choose a referent which the mentioned pronoun refers to. You will be given two options in each case and one choice should seem much more likely to you than the other.\n\nPositive Example 1 -\nInput: sentence: I couldn't put the saucepan on the rack because it was too tall .pronoun: it. A) the saucepan B) the rack\nOutput: the rack\n\nPositive Example 2 -\nInput: sentence: Arnold greatly influenced Jackson , though he lived two centuries earlier .pronoun: he. A) jackson B) arnold\nOutput: arnold\n\nNegative Example 1 -\nInput: sentence: Joe and David's uncles can still beat them at tennis , even though they are 30 years younger .pronoun: they. A) joe and david B) joe and david's uncles\nOutput: joe and david's uncles\n\nNegative Example 2 -\nInput: sentence: gaston passed the half-empty plate to hershel because he was full .pronoun: he. A) Gaston B) hershel\nOutput: hershel\n\nNow complete the following example -\nInput: sentence: jenna's mother had died long ago , and her education had been managed by an excellent woman as governess .pronoun: her. A) jenna B) jenna's mother\nOutput:",jenna,A) Jenna,0.666667,0.000000,0.666667
2,3,"Definition: In each example you will read a short sentence (or two). Then, a pronoun in the text is mentioned. Your task is to choose a referent which the mentioned pronoun refers to. You will be given two options in each case and one choice should seem much more likely to you than the other.\n\nPositive Example 1 -\nInput: sentence: I couldn't put the saucepan on the rack because it was too tall .pronoun: it. A) the saucepan B) the rack\nOutput: the rack\n\nPositive Example 2 -\nInput: sentence: Arnold greatly influenced Jackson , though he lived two centuries earlier .pronoun: he. A) jackson B) arnold\nOutput: arnold\n\nNegative Example 1 -\nInput: sentence: Joe and David's uncles can still beat them at tennis , even though they are 30 years younger .pronoun: they. A) joe and david B) joe and david's uncles\nOutput: joe and david's uncles\n\nNegative Example 2 -\nInput: sentence: gaston passed the half-empty plate to hershel because he was full .pronoun: he. A) Gaston B) hershel\nOutput: hershel\n\nNow complete the following example -\nInput: sentence: Look ! There are minnows swimming right below those ducks ! They had better get away to safety fast !pronoun: They. A) the minnows B) the ducks\nOutput:",the minnows,The ducks,0.500000,0.000000,0.500000
3,4,"Definition: In each example you will read a short sentence (or two). Then, a pronoun in the


Average ROUGE Scores for Our Model:
ROUGE-1 F1 Score: 0.4550
ROUGE-2 F1 Score: 0.1667
ROUGE-L F1 Score: 0.4550


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [35]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,
    lora_alpha=50,
    use_rslora=True,   # automatically sets adapting scaling factor based on lora_alpha to lora_alpha/math.sqrt(r), shd be better than original default
    layers_to_transform=list(range(24,32)),
    lora_dropout=0.05,
    bias="none",
    target_modules=[ # Llama 3.1-8B target modules
        "q_proj",      # 4096 -> 4096
        "k_proj",      # 4096 -> 1024
        "v_proj",      # 4096 -> 1024
        "gate_proj",   # 4096 -> 14336
        "up_proj",     # 4096 -> 14336
    ],
)

In [36]:
lora_model = get_peft_model(model, config)

In [37]:
lora_model.print_trainable_parameters()

trainable params: 7,077,888 || all params: 8,037,339,136 || trainable%: 0.0881


In [ ]:
lora_model.save_pretrained("llama3_1_8b-lora-pretraining")
# lora_model.save_pretrained("llama3_1_8b-lora-posttraining")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.